ETL - Transport Urbain : Chicago & Philadelphie
================================================
Projet : Dashboard Power BI - Analyse Ridership
Auteur : Data Analyst
Date : Février 2026

### Conversion et exploration des fichier Chicago (RDF) : Transformation du format RDF vers CSV via un script Python.

In [ ]:
# Importation des libraries nécessaires
import pandas as pd
import numpy as np
from scipy import stats
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import RDF, XSD


Conversion des fichiers RDF to CSV

In [15]:
RDF_FILES = [
    "C:\Python_challenges\Brief_jury_blanc\Chicago-Philadelphie\data-as-files-698094fa4d54e700584170\CTA Chicago - Ridership - Bus Routes - Daily Type Averages & Totals (RDF).rdf",
    "C:\Python_challenges\Brief_jury_blanc\Chicago-Philadelphie\data-as-files-698094fa4d54e700584170\CTA Chicago - Ridership - Bus Routes - Monthly Day-Type Averages & Totals (RDF).rdf"
]

In [8]:

g = Graph()

g.parse("C:\Python_challenges\Brief_jury_blanc\Chicago-Philadelphie\data-as-files-698094fa4d54e700584170\CTA Chicago - Ridership - Bus Routes - Daily Type Averages & Totals (RDF).rdf", format="xml")

print("Nombre de triplets :", len(g))


Nombre de triplets : 3500


In [5]:
predicates = set()

for s, p, o in g:
    predicates.add(p)

for p in list(predicates)[:20]:
    print(p)

In [9]:
g = Graph()
g.parse("C:\Python_challenges\Brief_jury_blanc\Chicago-Philadelphie\data-as-files-698094fa4d54e700584170\CTA Chicago - Ridership - Bus Routes - Monthly Day-Type Averages & Totals (RDF).rdf", format="xml")

print("Nombre de triplets :", len(g))

Nombre de triplets : 5000


In [10]:
predicates = set()

for s, p, o in g:
    predicates.add(p)

for p in list(predicates)[:20]:
    print(p)

http://www.socrata.com/rdf/terms#rowID
https://data.cityofchicago.org/resource/bynn-gwxy/avg_saturday_rides
https://data.cityofchicago.org/resource/bynn-gwxy/avg_sunday_holiday_rides
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
https://data.cityofchicago.org/resource/bynn-gwxy/avg_weekday_rides
http://www.w3.org/2000/01/rdf-schema#member
https://data.cityofchicago.org/resource/bynn-gwxy/monthtotal
https://data.cityofchicago.org/resource/bynn-gwxy/month_beginning
https://data.cityofchicago.org/resource/bynn-gwxy/route
https://data.cityofchicago.org/resource/bynn-gwxy/routename


In [16]:
RDF_DAILY = RDF_FILES[0]

NS_DAILY = {
    "route": URIRef("https://data.cityofchicago.org/resource/jyb9-n7fm/route"),
    "date": URIRef("https://data.cityofchicago.org/resource/jyb9-n7fm/date"),
    "day_type": URIRef("https://data.cityofchicago.org/resource/jyb9-n7fm/daytype"),
    "ridership": URIRef("https://data.cityofchicago.org/resource/jyb9-n7fm/rides")
}

g = Graph()
g.parse(RDF_DAILY, format="xml")
print("Triplets RDF (daily):", len(g))

records = {}

for s, p, o in g:
    if p in NS_DAILY.values():
        if s not in records:
            records[s] = {
                "date": None,
                "route": None,
                "day_type": None,
                "ridership": None
            }

        if p == NS_DAILY["date"]:
            records[s]["date"] = str(o)

        elif p == NS_DAILY["route"]:
            records[s]["route"] = str(o)

        elif p == NS_DAILY["day_type"]:
            records[s]["day_type"] = str(o)

        elif p == NS_DAILY["ridership"]:
            try:
                records[s]["ridership"] = int(o)
            except:
                records[s]["ridership"] = None

df_daily = pd.DataFrame.from_dict(records, orient="index")

# 🔹 Nettoyage
df_daily["date"] = pd.to_datetime(df_daily["date"], errors="coerce")
df_daily = df_daily.dropna(subset=["date", "route", "ridership"])
df_daily["route"] = df_daily["route"].str.upper().str.strip()
df_daily["day_type"] = df_daily["day_type"].str.capitalize()

# 🔹 Export CSV
OUTPUT_DAILY = r"C:\Python_challenges\Brief_jury_blanc\Chicago-Philadelphie\output\chicago_ridership_daily.csv"
df_daily.to_csv(OUTPUT_DAILY, index=False)

print("✅ CSV DAILY généré :", OUTPUT_DAILY)

Triplets RDF (daily): 3500
✅ CSV DAILY généré : C:\Python_challenges\Brief_jury_blanc\Chicago-Philadelphie\output\chicago_ridership_daily.csv


In [17]:
RDF_MONTHLY = RDF_FILES[1]

NS_MONTHLY = {
    "route": URIRef("https://data.cityofchicago.org/resource/bynn-gwxy/route"),
    "route_name": URIRef("https://data.cityofchicago.org/resource/bynn-gwxy/routename"),
    "avg_weekday": URIRef("https://data.cityofchicago.org/resource/bynn-gwxy/avg_weekday_rides"),
    "avg_saturday": URIRef("https://data.cityofchicago.org/resource/bynn-gwxy/avg_saturday_rides"),
    "avg_sunday": URIRef("https://data.cityofchicago.org/resource/bynn-gwxy/avg_sunday_holiday_rides"),
    "month_total": URIRef("https://data.cityofchicago.org/resource/bynn-gwxy/monthtotal"),
    "month_beginning": URIRef("https://data.cityofchicago.org/resource/bynn-gwxy/month_beginning")
}

g = Graph()
g.parse(RDF_MONTHLY, format="xml")
print("Triplets RDF (monthly):", len(g))

records = {}

for s, p, o in g:
    if p in NS_MONTHLY.values():
        if s not in records:
            records[s] = {
                "month_beginning": None,
                "route": None,
                "route_name": None,
                "avg_weekday_rides": None,
                "avg_saturday_rides": None,
                "avg_sunday_holiday_rides": None,
                "month_total": None
            }

        if p == NS_MONTHLY["month_beginning"]:
            records[s]["month_beginning"] = str(o)

        elif p == NS_MONTHLY["route"]:
            records[s]["route"] = str(o)

        elif p == NS_MONTHLY["route_name"]:
            records[s]["route_name"] = str(o)

        elif p == NS_MONTHLY["avg_weekday"]:
            records[s]["avg_weekday_rides"] = float(o)

        elif p == NS_MONTHLY["avg_saturday"]:
            records[s]["avg_saturday_rides"] = float(o)

        elif p == NS_MONTHLY["avg_sunday"]:
            records[s]["avg_sunday_holiday_rides"] = float(o)

        elif p == NS_MONTHLY["month_total"]:
            records[s]["month_total"] = float(o)

df_monthly = pd.DataFrame.from_dict(records, orient="index")

# 🔹 Nettoyage
df_monthly["month_beginning"] = pd.to_datetime(df_monthly["month_beginning"], errors="coerce")
df_monthly["route"] = df_monthly["route"].str.upper().str.strip()

# 🔹 Export CSV
OUTPUT_MONTHLY = r"C:\Python_challenges\Brief_jury_blanc\Chicago-Philadelphie\output\chicago_ridership_monthly.csv"
df_monthly.to_csv(OUTPUT_MONTHLY, index=False)

print("✅ CSV MONTHLY généré :", OUTPUT_MONTHLY)

Triplets RDF (monthly): 5000
✅ CSV MONTHLY généré : C:\Python_challenges\Brief_jury_blanc\Chicago-Philadelphie\output\chicago_ridership_monthly.csv


In [18]:
print("📊 Résultat global :")
print(df_monthly.head())
print(df_monthly.info())

📊 Résultat global :
                                                   month_beginning route  \
https://data.cityofchicago.org/resource/bynn-gw...      2001-04-01    47   
https://data.cityofchicago.org/resource/bynn-gw...      2001-01-01   204   
https://data.cityofchicago.org/resource/bynn-gw...      2001-01-01   203   
https://data.cityofchicago.org/resource/bynn-gw...      2001-03-01    49   
https://data.cityofchicago.org/resource/bynn-gw...      2001-02-01   147   

                                                             route_name  \
https://data.cityofchicago.org/resource/bynn-gw...                 47th   
https://data.cityofchicago.org/resource/bynn-gw...                Dodge   
https://data.cityofchicago.org/resource/bynn-gw...          Ridge/Grant   
https://data.cityofchicago.org/resource/bynn-gw...              Western   
https://data.cityofchicago.org/resource/bynn-gw...  Outer Drive Express   

                                                    avg_weekday_rides  \

### Exploration du fichier Chicago (Excel) : CTA Daily Boarding Totals (multi-mode : Bus, Rail, Total)

In [19]:
import pandas as pd

df_excel = pd.read_excel(
    "chicago_excel.xlsx",
    sheet_name=0
)

df_excel.head()

,service_date,day_type,bus,rail_boardings,total_rides
0,2001-01-01,U,297192,126455,423647
1,2001-01-02,W,780827,501952,1282779
2,2001-01-03,W,824923,536432,1361355
3,2001-01-04,W,870021,550011,1420032
4,2001-01-05,W,890426,557917,1448343


In [20]:
df_excel['service_date'] = pd.to_datetime(df_excel['service_date'])

In [23]:
df_excel = df_excel.dropna(subset=['service_date', 'bus', 'total_rides'])

In [24]:
df_excel['day_type'].value_counts()

W    6355
U    1449
A    1296
Name: day_type, dtype: int64

In [27]:
df_excel.duplicated(subset=['service_date']).sum()

0

In [28]:
df_excel['year'] = df_excel['service_date'].dt.year
df_excel['month'] = df_excel['service_date'].dt.month
df_excel['day'] = df_excel['service_date'].dt.day
df_excel['weekday'] = df_excel['service_date'].dt.day_name()

In [29]:
df_excel = df_excel[
    (df_excel['service_date'].dt.year >= 2019) &
    (df_excel['service_date'].dt.year <= 2025)
]

In [30]:
df_excel['service_date'].dt.year.value_counts().sort_index()

2019    365
2020    366
2021    365
2022    365
2023    365
2024    366
2025    334
Name: service_date, dtype: int64

In [32]:
df_excel.to_csv("output/cta_daily_ridership_clean.csv", index=False)

### Philadelphie (CSV) : Import et standardisation.

In [ ]:
import pandas as pd

file_route = "C:\\Python_challenges\\Brief_jury_blanc\\Chicago-Philadelphie\\data-as-files-698094fa4d54e700584170\\Average_Daily_Ridership_By_Route - City of Philadelphia.csv"
file_mode = "C:\\Python_challenges\\Brief_jury_blanc\\Chicago-Philadelphie\\data-as-files-698094fa4d54e700584170\\Average_Daily_Ridership_By_Mode - City of Philadelphia.csv"

df_route = pd.read_csv(file_route)
df_mode = pd.read_csv(file_mode)

### API Temps réel : CTA Bus Tracker (Chicago) , SEPTA TransitView (Philadelphie) ...etc

In [33]:
import requests

In [ ]:
def get_api_data(api_key):
    print("\n🌐 3. Récupération données API temps réel...")
    
    # CTA Chicago
    try:
        url = "http://www.ctabustracker.com/bustime/api/v3"
        params = {'key': api_key, 'rt': '20', 'format': 'json'}
        response = requests.get(url, params=params, timeout=5)
        
        if response.status_code == 200:
            df_cta = pd.DataFrame(response.json().get('bustime-response', {}).get('prd', []))
            print(f"✅ CTA API : {len(df_cta)} prédictions")
        else:
            df_cta = pd.DataFrame()
            print("⚠️  CTA API : Pas de données")
    except:
        df_cta = pd.DataFrame()
        print("⚠️  CTA API : Erreur de connexion")
    
    # SEPTA Philadelphia
    try:
        url = "http://www3.septa.org/api/TransitView/index.php"
        params = {'route': '23'}
        response = requests.get(url, params=params, timeout=5)
        
        if response.status_code == 200 and isinstance(response.json(), list):
            df_septa = pd.DataFrame(response.json())
            print(f"✅ SEPTA API : {len(df_septa)} véhicules")
        else:
            df_septa = pd.DataFrame()
            print("⚠️  SEPTA API : Pas de données")
    except:
        df_septa = pd.DataFrame()
        print("⚠️  SEPTA API : Erreur de connexion")
    
    return df_cta, df_septa


In [39]:
df_cta, df_septa = get_api_data("3cLAVZYxYrgAigNhEuQYBuBV8")


🌐 3. Récupération données API temps réel...
✅ CTA API : 0 prédictions
⚠️  SEPTA API : Pas de données
